In [4]:
import os 
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_astradb import AstraDBVectorStore
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
import cassio


load_dotenv()
os.environ['ASTRA_DB_APPLICATION_TOKEN']=os.getenv('ASTRA_DB_APPLICATION_TOKEN')
os.environ['ASTRA_DB_API_ENDPOINT'] = os.getenv('ASTRA_DB_API_ENDPOINT')
os.environ['HF_TOKEN'] =os.getenv('HF_TOKEN')
llm = ChatGroq(model='llama-3.3-70b-versatile')
embedding = HuggingFaceEndpointEmbeddings(model= 'sentence-transformers/all-MiniLM-L6-v2')

loader = PyPDFLoader('/Users/mohdsafeenkhan/Desktop/Machine_Learning/GEN-AI/14-RAG_with_AstraDB/attention.pdf')
doc = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 400)
doc_splitted = splitter.split_documents(doc)

astra_vector_db = AstraDBVectorStore(embedding=embedding,collection_name='sample_db')


astra_vector_db.add_documents(doc_splitted)


astra_db_retriver = astra_vector_db.as_retriever()


ChatPrompt = ChatPromptTemplate.from_template("""
You are a helpful assistant , provide the best answer you can ONLY WITH THE context provided , if you do not know the answer just say The context is not enough
<context>
{context}
<context>
Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm,ChatPrompt)
retrival_chain = create_retrieval_chain(astra_db_retriver,qa_chain)


response = retrival_chain.invoke(
    {'input':'what is self attention'}
)
response



APICommander about to raise from: [{'message': "Collection already exists: trying to create Collection ('sample_db') with different settings", 'errorCode': 'EXISTING_COLLECTION_DIFFERENT_SETTINGS', 'id': 'd4031ab4-25aa-48f7-b858-4c4252a2acb3', 'family': 'REQUEST', 'title': 'Collection already exists', 'scope': 'EMPTY'}]
/Users/mohdsafeenkhan/Desktop/Machine_Learning/GEN-AI/venv/lib/python3.11/site-packages/langchain_astradb/utils/astradb.py:448: UserWarning: Astra DB collection 'sample_db' is detected as having indexing turned on for all fields (either created manually or by older versions of this plugin). This implies stricter limitations on the amount of text each string in a document can store. Consider indexing anew on a fresh collection to be able to store longer texts. See https://github.com/langchain-ai/langchain-datastax/blob/main/libs/astradb/README.md#warnings-about-indexing for more details.
  if not self._validate_indexing_policy(


{'input': 'what is self attention',
 'context': [Document(id='5da45ed8a42841a7879d96b06691233f', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-08-03T00:07:29+00:00', 'author': '', 'keywords': '', 'moddate': '2023-08-03T00:07:29+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/mohdsafeenkhan/Desktop/Machine_Learning/GEN-AI/15-RAG_With_AstraDB/attention.pdf', 'total_pages': 15, 'page': 5, 'page_label': '6'}, page_content='versions produced nearly identical results (see Table 3 row (E)). We chose the sinusoidal version\nbecause it may allow the model to extrapolate to sequence lengths longer than the ones encountered\nduring training.\n4 Why Self-Attention\nIn this section we compare various aspects of self-attention layers to the recurrent and convolu-\ntional layers commonly used for mapping one variable-